#### Project 4: explore Stanford's school enrollment data

##### 1) create BQ dataset:

In [ ]:
dataset_id = "school_enrollments"

In [ ]:
!bq --location=US mk --dataset {dataset_id}

##### 2) create and populate BQ tables:

In [ ]:
import os

In [ ]:
gsutil_cmd = "gsutil ls gs://cs327e-open-access/school-enrollments/*"
file_listings = os.popen(gsutil_cmd)

for file in file_listings:

    start_index = file.rindex("_") + 1
    end_index = file.rindex(".")
    table = file[start_index:end_index]
    
    if table in ("co", "district", "fl", "ga", "id", "ma", "mo", "ne", "ny", "or", "school", "sd", "wi"):
        print("skipping " + table)
        continue

    bq_cmd = "bq --location=US load --autodetect --skip_leading_rows=1 "\
             "--source_format=CSV " + dataset_id + "." + table + ' ' + file
    print(bq_cmd)
    
    os.system(bq_cmd)

#### 3) Find school enrollments by school name, grade level, and year for Texas 

In [ ]:
!bq show --schema=true {dataset_id}.tx

#### Note: STRING type for "total" column

In [ ]:
%%bigquery
select school, grade, year, sum(cast(replace(replace(total, ",", ""), "<", "") as int64)) as total
from `cs327e-fa2021.school_enrollments.tx`
where total != "" and school != "" and district like '%AUSTIN%'
group by school, grade, year
order by school, grade, year

##### Note: BQ string functions documentation: 
##### https://cloud.google.com/bigquery/docs/reference/standard-sql/string_functions

In [ ]:
!bq --location=US mk --dataset "views"

In [ ]:
%%bigquery
create view views.austin_enrollments_by_school_grade_year as
select school, grade, year, sum(cast(replace(replace(total, ",", ""), "<", "") as int64)) as total
from `cs327e-fa2021.school_enrollments.tx`
where total != "" and school != "" and district like '%AUSTIN%'
group by school, grade, year
order by school, grade, year

In [ ]:
%%bigquery
select * from views.austin_enrollments_by_school_grade_year

#### 4) Open Data Studio and create chart from view:
##### https://datastudio.google.com/

#### Project 4 homework marker. Start homework from here. 